This code for defining the 2-Point Entropy Stable flux function. 

### Entropy Conservative (Ismail–Roe) Entropy-Conservative 2-Point Flux Function (2D Euler)

**Inputs: Left and right states**

$$
\begin{aligned}
\text{Left:} &\quad (\rho_L, u_{1,L}, u_{2,L}, p_L) \\
\text{Right:} &\quad (\rho_R, u_{1,R}, u_{2,R}, p_R)
\end{aligned}
$$

---

**1. Compute Temperatures**

$$
T_L = \frac{p_L}{R \rho_L}, \quad T_R = \frac{p_R}{R \rho_R}
$$

---

**2. Averaged Quantities**

- Velocity components:

$$
\hat{u}_j = \frac{u_{j,L} \sqrt{T_L} + u_{j,R} \sqrt{T_R}}{\sqrt{T_L} + \sqrt{T_R}}, \quad j = 1, 2
$$

- Pressure:

$$
\hat{p} = \frac{p_L \sqrt{T_L} + p_R \sqrt{T_R}}{\sqrt{T_L} + \sqrt{T_R}}
$$

- Density:

$$
\hat{\rho} = \frac{ \left( \frac{1}{\sqrt{T_L}} + \frac{1}{\sqrt{T_R}} \right)\left( \sqrt{T_L }\rho_L - \sqrt{T_R }\rho_R \right) }{ 2 \left( \log(\sqrt{T_L }\rho_L) - \log(\sqrt{T_R }\rho_R) \right) }
$$

- Specific Enthalpy:

$$
\hat{h} = \frac{\gamma}{\gamma - 1} \cdot \frac{\hat{p}}{\hat{\rho}}
$$

- Total Enthalpy:

$$
\hat{H} = \hat{h} + \frac{1}{2} \left( \hat{u}_1^2 + \hat{u}_2^2 \right)
$$

---

**3. Entropy-Conservative 2D Euler Flux in Direction** \( j = 1, 2 \)

$$
\mathbf{f}^S_j =
\begin{bmatrix}
\hat{\rho} \hat{u}_j \\
\hat{\rho} \hat{u}_j \hat{u}_1 + \delta_{j1} \hat{p} \\
\hat{\rho} \hat{u}_j \hat{u}_2 + \delta_{j2} \hat{p} \\
\hat{\rho} \hat{u}_j \hat{H}
\end{bmatrix}
$$

Where:

$$
\delta_{jk} =
\begin{cases}
1 & \text{if } j = k \\
0 & \text{otherwise}
\end{cases}
$$


In [12]:
import numpy as np
## The Ismail-Roe Entropy Conservative Flux
def EC_flux_IR(qA, qB, gamma=1.4, R=1.0, eps=1e-12):
    """
    Core 2-point Entropy Conservative (Ismail–Roe) entropy-conservative flux between two states qA, qB.
    qA, qB: arrays [rho, rho*u, rho*v, rho*E]
    Returns (flux_x, flux_y) each size-4 vectors.
    """
    rhoA, ruA, rvA, rEA = qA # Standard python un-packing
    rhoB, ruB, rvB, rEB = qB # Standard python un-packing

    # Primitive at A
    u1A, u2A = ruA/rhoA, rvA/rhoA
    EA = rEA/rhoA
    pA = (gamma-1)*(rEA - 0.5*rhoA*(u1A**2+u2A**2))
    # Primitive at B
    u1B, u2B = ruB/rhoB, rvB/rhoB
    EB = rEB/rhoB
    pB = (gamma-1)*(rEB - 0.5*rhoB*(u1B**2+u2B**2))

    # Temperatures
    TA, TB = pA/(R*rhoA), pB/(R*rhoB)
    sTA, sTB = np.sqrt(TA), np.sqrt(TB)

    # Denominator for averages
    den = 2*(np.log(sTA*rhoA) - np.log(sTB*rhoB))

    if abs(den) < eps:
        # States are nearly equal: return physical flux of state A
        fx = np.array([rhoA*u1A,
                       rhoA*u1A*u1A + pA,
                       rhoA*u1A*u2A,
                       u1A*(rhoA*EA + pA)])
        fy = np.array([rhoA*u2A,
                       rhoA*u2A*u1A,
                       rhoA*u2A*u2A + pA,
                       u2A*(rhoA*EA + pA)])
        return fx, fy

    # Averages
    u1h = (u1A*sTA + u1B*sTB)/(sTA + sTB)
    u2h = (u2A*sTA + u2B*sTB)/(sTA + sTB)
    ph  = (pA*sTA  + pB*sTB )/(sTA + sTB)
    num = (1/sTA + 1/sTB)*(sTA*rhoA-sTB*rhoB)
    rhoh = num/den
    hh  = gamma/(gamma-1)*(ph/rhoh)
    Hh  = hh + 0.5*(u1h**2 + u2h**2)

    # Flux components
    fx = np.array([rhoh*u1h,
                   rhoh*u1h*u1h + ph,
                   rhoh*u1h*u2h,
                   rhoh*u1h*Hh])
    fy = np.array([rhoh*u2h,
                   rhoh*u2h*u1h,
                   rhoh*u2h*u2h + ph,
                   rhoh*u2h*Hh])
    return fx, fy

def compute_fluxes_on_grid(q, gamma=1.4, R=1.0):
    """
    Compute Entropy Conservative fluxes on all interfaces of a 2D Euler grid `q`.
    q: array of shape (nx, ny, 4) holding [rho, rho*u, rho*v, rho*E]
    Returns:
      Fx: array shape (nx-1, ny, 4) of fluxes between (i,j) and (i+1,j)
      Fy: array shape (nx, ny-1, 4) of fluxes between (i,j) and (i,j+1)
    """
    nx, ny, _ = q.shape
    Fx = np.zeros((nx-1, ny, 4))
    Fy = np.zeros((nx, ny-1, 4))

    for i in range(nx-1):
        for j in range(ny):
            Fx[i,j], _ = EC_flux(q[i,j], q[i+1,j], gamma, R)

    for i in range(nx):
        for j in range(ny-1):
            _, Fy[i,j] = EC_flux(q[i,j], q[i,j+1], gamma, R)

    return Fx, Fy

# Example usage
# build a small grid of 3x3 nodes with uniform state
uniform_state = [1.0, 0.75, 0.0, 2.5]
q = np.tile(uniform_state, (3,3,1))

Fx, Fy = compute_fluxes_on_grid(q)
print("Fx shape:", Fx.shape, "Fy shape:", Fy.shape)
print("\nFx:\n", Fx)
print("\nFy:\n", Fy)


Fx shape: (2, 3, 4) Fy shape: (3, 2, 4)

Fx:
 [[[0.75     1.45     0.       2.540625]
  [0.75     1.45     0.       2.540625]
  [0.75     1.45     0.       2.540625]]

 [[0.75     1.45     0.       2.540625]
  [0.75     1.45     0.       2.540625]
  [0.75     1.45     0.       2.540625]]]

Fy:
 [[[0.     0.     0.8875 0.    ]
  [0.     0.     0.8875 0.    ]]

 [[0.     0.     0.8875 0.    ]
  [0.     0.     0.8875 0.    ]]

 [[0.     0.     0.8875 0.    ]
  [0.     0.     0.8875 0.    ]]]


In [ ]:


def Ec_flux_P(qA, qB, gamma=1.4, R=1.0, eps=1e-12):
    """
    Entropy‑conservative 2D Euler flux using Parsani’s 'hat' averages,
    Inputs qA, qB = [rho, rho*u, rho*v, rho*E].
    Returns (fx, fy), each a 4‑vector.
    """
    # Note: H = h + (1/2)*(u_j*u_j), and recall the invscid Flux \rho*u_j*H 
    # 1) unpack conserv. vars
    rhoA, ruA, rvA, rEA = qA
    rhoB, ruB, rvB, rEB = qB

    # 2) primitives at A
    u1A = ruA/rhoA; u2A = rvA/rhoA
    EA = rEA/rhoA
    pA = (gamma-1)*(rEA - 0.5*rhoA*(u1A**2 + u2A**2))
    hA = EA + pA/rhoA
    # primitives at B
    u1B = ruB/rhoB; u2B = rvB/rhoB
    EB = rEB/rhoB
    pB = (gamma-1)*(rEB - 0.5*rhoB*(u1B**2 + u2B**2))
    hB = EB + pB/rhoB

    # 3) T and its inverse‐sqrt
    TA = pA/(R*rhoA)
    TB = pB/(R*rhoB)
    iTA = 1.0/np.sqrt(TA)
    iTB = 1.0/np.sqrt(TB)

    # if temperatures nearly equal, fallback
    den = iTA + iTB
    if den < eps:
        # physical flux of A
        fx = np.array([
            rhoA*u1A,
            rhoA*u1A*u1A + pA,
            rhoA*u1A*u2A,
            rhoA*u1A*(EA + pA/rhoA)
        ])
        fy = np.array([
            rhoA*u2A,
            rhoA*u2A*u1A,
            rhoA*u2A*u2A + pA,
            rhoA*u2A*(EA + pA/rhoA)
        ])
        return fx, fy

    # 4) hat‐averages
    u1h = (u1A*iTA + u1B*iTB) / den
    u2h = (u2A*iTA + u2B*iTB) / den
    ph  = (  pA*iTA +   pB*iTB) / den
    theta1 = (rhoA*np.sqrt(TA)+rhoB*np.sqrt(TB))/((iTA+iTB)*(rhoA*np.sqrt(TA)-rhoB*np.sqrt(TB)))
    theta2 = ((gamma + 1)/(gamma - 1))*((np.log(np.sqrt(TB/TA)))/(np.log((rhoA/rhoB)*np.sqrt(TA/TB)*(iTA - iTB))))

    hh  = ((  np.log((np.sqrt(TA)*rhoA)/(np.sqrt(TB)*rhoB))) / den)*(theta1+theta2)
    Hh  = hh + 0.5*(u1h**2 + u2h**2)

    # density‐hat: ρ̂ = (iTA+iTB)*(ρA*iTA - ρB*iTB) / [2*(ln(ρA*iTA)-ln(ρB*iTB))]
    num = (iTA + iTB)*(rhoA*iTA - rhoB*iTB)
    den2 = 2*(np.log(rhoA*iTA) - np.log(rhoB*iTB))
    rhoh = num/den2

    # 5) build fluxes
    fx = np.array([
        rhoh*u1h,
        rhoh*u1h*u1h + ph,
        rhoh*u1h*u2h,
        rhoh*u1h*Hh
    ])
    fy = np.array([
        rhoh*u2h,
        rhoh*u2h*u1h,
        rhoh*u2h*u2h + ph,
        rhoh*u2h*Hh
    ])

    return fx, fy


In [19]:
# Test 1: nontrivial states
qA = np.array([1.0, 0.5, 0.2, 2.5])
qB = np.array([1.0, -0.3, 0.4, 2.3])
fx, fy = Ec_flux_P(qA, qB)
print("fx =", fx)
print("fy =", fy)

# Test 2: identical states (fallback)
qC = np.array([1.0, 0.0, 0.0, 2.5])
fx2, fy2 = Ec_flux_P(qC, qC)
print("fx2 =", fx2)
print("fy2 =", fy2)

fx = [0.10170736 0.91464663 0.03071435 0.34103816]
fy = [0.33367091 0.03071435 1.00604894 1.11884252]
fx2 = [nan nan nan nan]
fy2 = [nan nan nan nan]


/var/folders/1m/5m1555qx5p7chjhs65rc82dm6hph4m/T/ipykernel_9551/2168978670.py:56: RuntimeWarning: invalid value encountered in double_scalars
  rhoh = num/den2


Comments: 

I noticed that when the states are close to each other, the flux function fails to evaluate a value. Therefore, for smooth flows, this could pose an issue. 
It seems that there are multiple ways in which a flux function can be constructed. 